In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
import os
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
os.environ["GROQ_API_KEY"]="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"

In [ ]:
# Loading the doocuments
loader=UnstructuredFileLoader("/content/attention all you need.pdf")
documents=loader.load()

In [ ]:
print(documents)

[Document(metadata={'source': '/content/attention all you need.pdf'}, page_content='3 2 0 2\n\ng u A 2\n\n] L C . s c [\n\n7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and dec

In [ ]:
# Splitting text into Text chunks

text_splitter=CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
    )

In [ ]:
text=text_splitter.split_documents(documents)

In [ ]:
type(text)

list

In [ ]:
len(text)

22

In [ ]:
text[0:2]

[Document(metadata={'source': '/content/attention all you need.pdf'}, page_content='3 2 0 2\n\ng u A 2\n\n] L C . s c [\n\n7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a\n\nProvided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\nAttention Is All You Need\n\nAshish Vaswani∗ Google Brain avaswani@google.com\n\nNoam Shazeer∗ Google Brain noam@google.com\n\nNiki Parmar∗ Google Research nikip@google.com\n\nJakob Uszkoreit∗ Google Research usz@google.com\n\nLlion Jones∗ Google Research llion@google.com\n\nAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu\n\nŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com\n\nIllia Polosukhin∗ ‡ illia.polosukhin@gmail.com\n\nAbstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and dec

In [ ]:
embedding=HuggingFaceEmbeddings() # Loading the default hugging face embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory="doc_db"

In [ ]:
vectordb=Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
# Retriver

retriver=vectordb.as_retriever()

In [ ]:
llm=ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [ ]:
# Creating a QA chain
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriver,
    return_source_documents=True
)

In [ ]:
# invoke the QA chain and get a response for user query

query="Who is the model archietecture discussed in this paper?"
response=qa_chain.invoke({"query": query})

In [ ]:
print(response["result"])

The model architecture discussed in this paper is called the Transformer.


In [ ]:
print(response["source_documents"][0].metadata["source"])

/content/attention all you need.pdf


In [ ]:
query="What is the concept mentioned in this"
response=qa_chain.invoke({"query": query})
print(response["result"])
print('*')
print(response["source_documents"][0].metadata["source"])

The concept mentioned in this text is primarily about Attention Mechanisms in Neural Networks, specifically in the context of Sequence-to-Sequence Learning and Neural Machine Translation.
******************************
/content/attention all you need.pdf
